# Projeto 1 - Ciência dos Dados

Nome: Pedro Herique Britto Aragão Andrade

Nome: Thiago Shiguero Kawahara

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\thiag\Projeto1-Ciedados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Pringles.xlsx'

In [4]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')

train.Classificação = train.Classificação.astype('category')
train.Classificação.cat.categories = (['Irrelevante', 'Relevante'])

train.head(5)

,Treinamento,Classificação
0,eu 🤝 comer pringles com danette\n\n combinaç...,Relevante
1,"@englantinez pipoca, bolinha de amendoim ou ba...",Relevante
2,chorando e comendo pringles,Irrelevante
3,leticia podendo me viciar em tanta coisa me vi...,Relevante
4,vontade de uma pringles ughhhhh,Irrelevante


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')

test.Classificação = test.Classificação.astype('category')
test.Classificação.cat.categories = (['Irrelevante', 'Relevante'])

test.head(5)

,Teste,Classificação
0,o cara tem um pote de pringles entre as perna ...,Irrelevante
1,@ellixquintx a pringles de cebola maior que ca...,Irrelevante
2,se eu fosse líder eu n ia falar com a pessoa q...,Relevante
3,vou fazer meu vício favorito dos últimos tempo...,Relevante
4,ai to muito puta namoral ja era a minha coca c...,Relevante


___
## Classificador automático de sentimento


Pringles é um aperitivo estilo batata chips e vendido em um tubo cilindrico.

Relevante: Avaliações negativas do produto ou que tivessem informações interessantes para uma possível parceria ou novas ideias de marketing.

Irrelevante: Todos que não fossem relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Utiliza-se o Naive-Bayes para calcular a probabilidade do Tweet ser Relevante e a probabilidade do Tweet ser Irrelevante, depois essas probabilidades são comparadas (i) e a partir do resultado é definido se o Tweet é relevante ou irrelevante.

i.$\quad  P(R|T) > P(I|T)$ ou $P(R|T) < P(I|T)$ $\quad$ **ETAPA 5**

Portanto, temos que descobrir: 

* $P(R|T)$ 

* $P(I|T)$

As etapas a seguir valem tanto para os **Relevantes** quanto para os **Irrelevantes**.

Sabemos que:

$\quad P(R|T) = \frac{P(TintersecçãoR)}{P(T)}$ $\Rightarrow$ $P(R|T) = \frac{P(T|R) \cdot P(R)}{P(T)}$ $\quad$ **ETAPA 4**

Assim, devemos calcular:

* $P(T|R)$ $\quad$ **ETAPA 3**

$\quad\quad P(T|R) = P(palavra1|R) \cdot P(palavra2|R) \cdot P(palavra3|R) \cdot ... \cdot P(palavraN|R)$

$\quad\quad\quad P(palavra|R) = \frac{x}{y}$

$\quad\quad\quad x=$ total de vezes que a palavra apareceu nos tweets relevantes

$\quad\quad\quad y=$ total de palavras dos tweets relevantes

* $P(R)$

$\quad\quad P(R) =$ $\frac{y}{w}$ $\quad$ **ETAPA 2** 

$\quad\quad w=$ total de palavras de todos os tweets 

$\quad$ Para encontrar $y$ e $w$: $\quad$ **ETAPA 1**

$\quad\quad$ - Separar o DataFrame em Relevante e Irrelevante
 
$\quad\quad$ - Contar quantas palavras tem nos tweets Relevantes, Irrelevantes e Total (Relevantes + Irrelevantes)

* Não é preciso calcular $P(T)$, pois esse é cortado na equação de comparação (i).

**LIMPEZA**

Utilizamos duas funções de limpeza (limpandinho() e emoji()) para aumentar a precisão do classificador.

In [6]:
# funcao de limpeza 

def limpandinho(text):
    
    # palavras com suas versoes + acentucao de letras
    dicio = {"nao": ["n","nn","nnn"], 
             "sim": ["ss","s","sss","siim"], 
             "porque": ["pq","motivo"],
             "muito": ["mt"],
             "tudo": ["td","todo"],
             "a": ["ã","á","à","â"],
             "e": ["è","é","ê"],
             "i": ["ì","í","î"],
             "o": ["õ","ò","ó","ô"],
             "u": ["ù","ú","û"],
             "c": ["ç"]
            }

    valores=[]
    for lista in dicio.values():
        for elemento in lista:
            valores.append(elemento)

    # pega cada LETRA da frase e tira acentuacao (i) + pontuacao(ii)
    l=[]
    l2= []
    for letra in text:
        l.append(letra)
    for letra in l:
        if letra in valores:
            for l_dicio in dicio.values():
                # (i)
                if letra in l_dicio:
                    l2.append(list(dicio.keys())[list(dicio.values()).index(l_dicio)])
        # (ii)
        elif letra in ["'",'"',"|","!","-",".","/",":","?",";",",",")","(","¿"]:
            pass
                    
        else:
            l2.append(letra)
            
    frase = "".join(l2)
    l_palavras = frase.split()
          
    # pega cada PALAVRA e transforma as palavras abreviadas em seus padroes (iii) + deleta palavras sem valor (iv)
    nova_frase=[]  
    for palavra in l_palavras:
        if palavra in valores:
            for l_dicio in dicio.values():
                # (iii)
                if palavra in l_dicio:
                    nova_frase.append(list(dicio.keys())[list(dicio.values()).index(l_dicio)])     
        elif palavra[:2] == "kk" or palavra=="k":
            palavra = "kkk"
            nova_frase.append(palavra)
            
        # (iv)    
        elif palavra[0] == "@":
            pass
        elif palavra[:4] == "http":
            pass
        elif palavra in ["a","o","de","era","em","que","q","oq"]:
            pass
        
        else:
            nova_frase.append(palavra)
    
    # retorna a frase ja chamando a funcao emoji()
    return emoji(" ".join(nova_frase))


In [7]:
# funcao de limpeza 

# garante que aja espacamento antes e depois de cada emoji
def emoji(frase):
    l= []
    for palavra in frase:
        for letra in palavra:
            l.append(letra)

    f= ""
    for n in range(len(l)):
        codigo = l[n].encode("utf-8")

        if len(codigo) != len(l[n]):

            if n == len(l)-1:
                if l[n-1] != " ":  f+= " " + l[n]
                else: f+= l[n]
            elif n == 0:
                if l[n+1] != " ":  f+= l[n] + " "
                else: f+= l[n]

            elif l[n-1] == " " and l[n+1] == " ":  f+= l[n]

            elif l[n-1] != " " and l[n+1] == " ": f+= " " + l[n]

            elif l[n-1] == " " and l[n+1] != " ": f+= l[n] + " "

            elif l[n-1] != " " and l[n+1] != " ": f+= " " + l[n] + " "

        else: f+= l[n]
            
    return f

**ETAPA 1**

$y=$ total de palavras dos tweets relevantes e irrelevantes

$w=$ total de palavras de todos os tweets 

In [8]:
# separando a tabela de treinamento em relevante e irrelevante 

train_i = train.loc[train.Classificação=="Irrelevante",:]

train_r = train.loc[train.Classificação=="Relevante",:]

In [9]:
# limpando, juntando todas as frases da tabela treinamento e individualizando cada palavra

# para tabela total
l_t=[]
for i in train.Treinamento:
    l_t.append(limpandinho(i))
l_t = " ".join(l_t).split()


# para tabela somente relevante
l_r=[]
for i in train_r.Treinamento:
    l_r.append(limpandinho(i))   
l_r = " ".join(l_r).split()


# para tabela somente irrelevante 
l_i=[]
for i in train_i.Treinamento:
    l_i.append(limpandinho(i))    
l_i = " ".join(l_i).split()

In [10]:
# contagem das palavras

# para tabela total
tab_total = pd.Series(l_t)
tab_total_a = tab_total.value_counts()
tab_total_r = tab_total.value_counts(True)

# para tabela somente relevante
tab_rel = pd.Series(l_r)
tab_rel_a = tab_rel.value_counts()
tab_rel_r = tab_rel.value_counts(True)

# para tabela somente irrelevante 
tab_irr = pd.Series(l_i)
tab_irr_a = tab_irr.value_counts()
tab_irr_r = tab_irr.value_counts(True)

**ETAPA 2** 

$P(R)$ e $P(I)$

In [11]:
# probabilidade de ser relevante
probR = sum(tab_rel_a)/sum(tab_total_a)

# probabilidade de ser irrelevante
probI = sum(tab_irr_a)/sum(tab_total_a)

**ETAPA 3**

$P(T|R)$ e $P(T|I)$

In [12]:
# todos os tipos de palavras que aparecem na tabela treinamento (para Laplace smoothing)

palavras_unicas_total=[]
for i in l_t:
    if i not in palavras_unicas_total:
        palavras_unicas_total.append(i)

In [13]:
# limpando e separando cada frase da tabela teste

l_teste=[]
for i in test.Teste:
    c = limpandinho(i)
    l_teste.append(c.split())

In [14]:
# criando um DataFrame para o computador preencher

data = {'Tweet':  [],
        'Classificação': [],
        }

novo = pd.DataFrame(data, columns = ['Tweet','Classificação'])

**ETAPA 3, 4 e 5**

In [15]:
# Legenda: T == Tweet (ex: probTDadoR == prob Tweet dado Relevante)

# contador para acessar a linha desejada
contador = 0
for frase in l_teste:
    
    # ETAPA 3  P(T|R) e P(T|I)
    probTDadoR = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_rel_a:
            probTDadoR*=(tab_rel_a[palavra] + 1) / (len(tab_rel) + len(palavras_unicas_total))
        else:
            probTDadoR*=(0 + 1) / (len(tab_rel) + len(palavras_unicas_total))
                    
    probTDadoI = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_irr_a:
            probTDadoI*=(tab_irr_a[palavra] + 1) / (len(tab_irr) + len(palavras_unicas_total))
        else:
            probTDadoI*=(0 + 1) / (len(tab_irr) + len(palavras_unicas_total))
      
    # ETAPA 4  P(R|T) e P(I|T)
    probRdadoT = probTDadoR*probR
    
    probIdadoT = probTDadoI*probI
 
    # ETAPA 5
    # comparacao das probabilidades + montando a tabela
    if probRdadoT>probIdadoT:
        
        novo.loc[contador,"Tweet"]= " ".join(frase)
        novo.loc[novo.Tweet==" ".join(frase),"Classificação"] = "Relevante"
    
    elif probRdadoT<probIdadoT:
        
        novo.loc[contador,"Tweet"]=" ".join(frase)
        novo.loc[novo.Tweet==" ".join(frase),"Classificação"] = "Irrelevante"
    
    contador+=1

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [16]:
# criando nova coluna 
novo["Performance"] = "nada"

# verificando e preenchendo a tabela
for i in range(len(novo)):
    if novo.iloc[i, 1] == test.iloc[i,1]:
        if novo.iloc[i, 1] == "Relevante": novo.iloc[i, 2]="VP"
            
        else: novo.iloc[i, 2]="VN"  
    
    elif novo.iloc[i, 1] != test.iloc[i,1]:       
        if novo.iloc[i, 1] == "Relevante": novo.iloc[i, 2]="FP"
            
        else: novo.iloc[i, 2]="FN"            

In [40]:
# calculo das porcentagens

P = novo.Performance.value_counts(normalize=True).round(4)*100
vp = P["VP"]
fp = P["FP"]
vn = P["VN"]
fn = P["FN"]
print(f"Porcentagem de verdadeiros positivos: {vp:.1f}")
print(f"Porcentagem de falsos positivos: {fp:.1f}")
print(f"Porcentagem de verdadeiros negativos: {vn:.1f}")
print(f"Porcentagem de falsos negativos: {fn:.1f}")

Porcentagem de verdadeiros positivos: 15.0
Porcentagem de falsos positivos: 16.0
Porcentagem de verdadeiros negativos: 55.0
Porcentagem de falsos negativos: 14.0


__________
### Concluindo

In [18]:
pd.crosstab(novo.Performance, novo.Classificação, margins= True)

Classificação,Irrelevante,Relevante,All
Performance,,,
FN,28,0,28
FP,0,32,32
VN,110,0,110
VP,0,30,30
All,138,62,200


In [19]:
pd.crosstab(novo.Performance, novo.Classificação, margins= True, normalize=True).round(4)*100

Classificação,Irrelevante,Relevante,All
Performance,,,
FN,14.0,0.0,14.0
FP,0.0,16.0,16.0
VN,55.0,0.0,55.0
VP,0.0,15.0,15.0
All,69.0,31.0,100.0


In [20]:
pd.crosstab(novo.Performance, novo.Classificação, margins= True, normalize="columns").round(4)*100

Classificação,Irrelevante,Relevante,All
Performance,,,
FN,20.29,0.00,14.0
FP,0.00,51.61,16.0
VN,79.71,0.00,55.0
VP,0.00,48.39,15.0


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

In [21]:
# criando um DataFrame

data = {'Tweet':  [],
        'Classificação': [],
        }

tetra = pd.DataFrame(data, columns = ['Tweet','Classificação'])

In [22]:
# criando um DataFrame com tweets de test + train

contador= 0
while True:
    try:
        tetra.loc[contador,"Tweet"]= test.loc[contador,"Teste"]
        tetra.loc[contador,["Classificação"]]= test.loc[contador,["Classificação"]]
    except:
        try:
            tetra.loc[contador,"Tweet"]= train.loc[contador-200,"Treinamento"]
            tetra.loc[contador,["Classificação"]]= train.loc[contador-200,["Classificação"]]
        except:
            break
    contador+= 1

In [23]:
# validacao

for i in range(len(tetra)):
    if i< 200:
        if tetra.iloc[i,0] == test.iloc[i,0]:
            pass
        else:
            print("fudeu",i)
    else:
        if tetra.iloc[i,0] == train.iloc[i-200,0]:
            pass
        else:
            print("fudeu",i)

In [24]:
test2 = []
train2 = []
clas_test= []
clas_train= []

numeros = []
for n in range(500):
    numeros.append(n)

while True:
    if len(numeros)== 0:
        break
    random.shuffle(numeros)
    test2.append(tetra.iloc[numeros[0],0])
    clas_test.append(tetra.iloc[numeros[0],1])

    train2.append(tetra.iloc[numeros[1],0])
    clas_train.append(tetra.iloc[numeros[0],1])
    
    numeros.remove(numeros[0])
    numeros.remove(numeros[0])

In [25]:
# validador

for i in range (len(test2)-1):
    if test2[i] in train2:
        print("fudeu", i)

In [26]:
data = {'Tweet':  [],
        'Classificação': [],
        }

train22 = pd.DataFrame(data, columns = ['Tweet','Classificação'])

In [27]:
data = {'Tweet':  [],
        'Classificação': [],
        }

test22 = pd.DataFrame(data, columns = ['Tweet','Classificação'])

In [28]:
contador = 0
while True:
    try:
        train22.loc[contador,"Tweet"]= train2[contador]
        train22.loc[contador,"Classificação"]= clas_train[contador]
        
        test22.loc[contador,"Tweet"]= test2[contador]
        test22.loc[contador,"Classificação"]= clas_test[contador]
    except:
        try:
            test22.loc[contador,"Tweet"]= test2[contador]
            test22.loc[contador,"Classificação"]= clas_test[contador]
        except:
            break
    contador+=1

In [29]:
train22_r = train22.loc[train22.Classificação=="Relevante",:]

train22_irr = train22.loc[train22.Classificação=="Irrelevante",:]

In [30]:
l_train22_t=[]
for i in train22.Tweet:
    l_train22_t.append(limpandinho(i))
l_train22_t = " ".join(l_train22_t).split()

l_train22_r=[]
for i in train22_r.Tweet:
    l_train22_r.append(limpandinho(i))
l_train22_r = " ".join(l_train22_r).split()

l_train22_irr=[]
for i in train22_irr.Tweet:
    l_train22_irr.append(limpandinho(i))
l_train22_irr = " ".join(l_train22_irr).split()

In [31]:
tab_train22_t = pd.Series(l_train22_t)
tab_train22_t_a = tab_train22_t.value_counts()
tab_train22_t_r = tab_train22_t.value_counts(True)

tab_train22_rel = pd.Series(l_train22_r)
tab_train22_rel_a = tab_train22_rel.value_counts()
tab_train22_rel_r = tab_train22_rel.value_counts(True)

tab_train22_irr = pd.Series(l_train22_irr)
tab_train22_irr_a = tab_train22_irr.value_counts()
tab_train22_irr_r = tab_train22_irr.value_counts(True)

In [32]:
probR = sum(tab_train22_rel_a)/sum(tab_train22_t_a)
probI = sum(tab_train22_irr_a)/sum(tab_train22_t_a)

In [33]:
data = {'Tweet':  [],
        'Classificação': [],
        }

novo2 = pd.DataFrame(data, columns = ['Tweet','Classificação'])

In [34]:
palavras_unicas_total=[]
for i in l_train22_t:
    if i not in palavras_unicas_total:
        palavras_unicas_total.append(i)

In [35]:
l_teste2=[]
for i in test22.Tweet:
    c = limpandinho(i)
    l_teste2.append(c.split())

In [36]:
# Legenda: T == Tweet

contador = 0
for frase in l_teste2:
    probTDadoR = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_train22_rel_a:
            probTDadoR*=(tab_train22_rel_a[palavra] + 1) / (len(tab_train22_rel) + len(palavras_unicas_total))
        else:
            probTDadoR*=(0 + 1) / (len(tab_train22_rel) + len(palavras_unicas_total))
            
            
    probTDadoI = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_train22_irr_a:
            probTDadoI*=(tab_train22_irr_a[palavra] + 1) / (len(tab_train22_irr) + len(palavras_unicas_total))
        else:
            probTDadoI*=(0 + 1) / (len(tab_train22_irr) + len(palavras_unicas_total))
            
    probRdadoT = probTDadoR*probR
    
    probIdadoT = probTDadoI*probI

    if probRdadoT>probIdadoT:
        
        novo2.loc[contador,"Tweet"]= " ".join(frase)
        novo2.loc[novo2.Tweet==" ".join(frase),"Classificação"] = "Relevante"
    
    elif probRdadoT<probIdadoT:
        
        novo2.loc[contador,"Tweet"]=" ".join(frase)
        novo2.loc[novo2.Tweet==" ".join(frase),"Classificação"] = "Irrelevante"
    
    contador+=1

In [37]:
novo2["Performance"] = "nada"

for i in range(len(novo2)):
    if novo2.iloc[i, 1] == test22.iloc[i,1]:
        if novo2.iloc[i, 1] == "Relevante": novo2.iloc[i, 2]="VP"
            
        else: novo2.iloc[i, 2]="VN"  
    
    elif novo2.iloc[i, 1] != test22.iloc[i,1]:       
        if novo2.iloc[i, 1] == "Relevante": novo2.iloc[i, 2]="FP"
            
        else: novo2.iloc[i, 2]="FN"    

In [38]:
P = novo2.loc[:,["Performance"]].value_counts(normalize=True).round(4)*100
vp = P["VP"]
fp = P["FP"]
vn = P["VN"]
fn = P["FN"]
print(f"Porcentagem de verdadeiros positivos: {vp:.1f}")
print(f"Porcentagem de falsos positivos: {fp:.1f}")
print(f"Porcentagem de verdadeiros negativos: {vn:.1f}")
print(f"Porcentagem de falsos negativos: {fn:.1f}")


Porcentagem de verdadeiros positivos: 9.2
Porcentagem de falsos positivos: 18.4
Porcentagem de verdadeiros negativos: 50.8
Porcentagem de falsos negativos: 21.6


In [39]:
test22

,Tweet,Classificação
0,troco qualquer ovo de páscoa do mundo por uma ...,Irrelevante
1,ai que vontade de comer pringles e doritos com...,Relevante
2,@rapbyunnie queria batatinha pringles e sorvet...,Relevante
3,"@caixadoremedios @carolborgesart ora ora, lite...",Irrelevante
4,comendo uma pringles e bebendo um vinho branco...,Relevante
...,...,...
245,a minha mae: queres que faça uma tarte de fran...,Irrelevante
246,boa promoção pra experimentar a pringles da ru...,Irrelevante
247,vontade de comer um pringles,Irrelevante
248,parte ruim de comer pringles é quando minha mã...,Relevante


___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* [FEITO] Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* [FEITO] Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* [FEITO] Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**